# **SCIENCE-U: Exploring sustainable fuel options with computer simulations**

![image.png](https://ecos-appdev-production.s3.amazonaws.com/science_site/s3fs-public/inline-images/OutreachSciUBanner2TransparentBackground_0.png)

# Authors: Seda Oturak and Simon Gelin

© [MOSAIC - Dabo Research Group, Pennsylvania State University](http://dabo.matse.psu.edu) (2023)

# Install simulation environment and simulation code

In [ ]:
# install ASE (the atomic simulation environment) using Linux
!apt install ase

# install GPAW (the simulation code) using Linux
!apt install python3-mpi4py cython3 libxc-dev gpaw-data
!pip -q install gpaw pymatgen==2023.5.10

In [ ]:
# import tools from ASE
from ase import Atoms, Atom
from ase.build import molecule as ase_molecule
from ase.data.pubchem import pubchem_atoms_search
from ase.visualize import view

# import tools from GPAW
from gpaw import GPAW, PW

# import tools for plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# import tools for numerical calculations
import numpy as np

# 1. Create the molecule &#129337;

In [ ]:
"""
define molecule using the PubChem database of the National Library of Medecine

methane         = 297
propane         = 6334
ethanol         = 702
octane          = 10907
oxygen          = 977
carbon dioxide  = 280
water           = 962
hydrogen        = n/a   (type the command "molecule = ase_molecule('H2')"
                        instead of "molecule = pubchem_atoms_search(cid=XXX)")

"""
molecule = pubchem_atoms_search(cid=297)
molecule.set_cell([7, 7, 7], scale_atoms=False)

# 2. Visualize the molecule &#x1F50D;

In [ ]:
# show molecule
view(molecule, viewer='x3d', block=True)

# 3. Calculate the energy of the molecule &#x1F4BB;

In [ ]:
# set simulation parameters
calc = GPAW(xc='PBE',
            kpts=(1,1,1),
            mode=PW(700),
            basis='dzp')
molecule.center()
molecule.set_calculator(calc)

# run computer simulation (using quantum mechanics to compute the electron cloud)
energy = molecule.get_total_energy()

print("*****")
print(" The energy of the molecule is "+str(energy)+" eV")
print("*****")

# 4. Visualize the electron cloud of the molecule &#x269B;

In [ ]:
# convert electron cloud
n = calc.get_all_electron_density(gridrefinement=4)
nred = n[::3,::3,::3]
X, Y, Z = np.mgrid[0:7:60j, 0:7:60j, 0:7:60j]

# show electron cloud
fig = go.Figure(data=go.Isosurface(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=nred.flatten(),
    isomin=0.1,
    isomax=1,
    surface_count=5,
    colorbar_nticks=5,
    caps=dict(x_show=False, y_show=False),
    opacity=0.3
    ))

fig.show()